In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Génération d'une série de prix fictive
dates = pd.date_range('2022-01-01', periods=60)
prix = np.concatenate([np.linspace(10, 20, 30), np.linspace(20, 15, 30)])
# Ajout d'un faux signal (bruit)
prix[40:45] += 2

df = pd.DataFrame({'Date': dates, 'Close': prix})
df = df.set_index('Date')

# Calcul EMA12 et EMA26
df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()
# Calcul MACD et MACD_signal
df['MACD'] = df['EMA12'] - df['EMA26']
df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

df[['Close', 'EMA12', 'EMA26', 'MACD', 'MACD_signal']].head(15)

## Visualisation du prix, MACD et MACD_signal

- Les croisements MACD/MACD_signal donnent des signaux d'achat/vente.
- On va aussi repérer une divergence.

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Prix et EMAs
axs[0].plot(df.index, df['Close'], label='Prix', color='black')
axs[0].plot(df.index, df['EMA12'], label='EMA12', color='blue', linestyle='--')
axs[0].plot(df.index, df['EMA26'], label='EMA26', color='red', linestyle='--')
axs[0].set_title('Prix et Moyennes Mobiles Exponentielles')
axs[0].legend()

# MACD et signal
axs[1].plot(df.index, df['MACD'], label='MACD', color='purple')
axs[1].plot(df.index, df['MACD_signal'], label='MACD_signal', color='orange')
axs[1].axhline(0, color='grey', linestyle=':')
axs[1].set_title('MACD et MACD_signal')
axs[1].legend()

plt.tight_layout()
plt.show()

## Détection d'une divergence

On cherche une zone où le prix fait un plus bas mais le MACD fait un plus haut (divergence haussière).

In [ ]:
# Recherche d'une divergence haussière
div_start, div_end = 28, 35
prix_zone = df['Close'].iloc[div_start:div_end]
macd_zone = df['MACD'].iloc[div_start:div_end]

print('Prix min:', prix_zone.min(), 'MACD min:', macd_zone.min())
print('Prix max:', prix_zone.max(), 'MACD max:', macd_zone.max())

plt.figure(figsize=(10,4))
plt.plot(prix_zone.index, prix_zone, label='Prix')
plt.plot(macd_zone.index, macd_zone, label='MACD')
plt.title('Exemple de divergence haussière')
plt.legend()
plt.show()

## Illustration d'un faux signal

On regarde une zone où le MACD croise à la hausse, mais le prix ne monte pas vraiment ensuite (faux signal).

In [ ]:
# Faux signal autour de l'indice 42
faux_signal_idx = 42

plt.figure(figsize=(10,4))
plt.plot(df.index, df['Close'], label='Prix')
plt.axvline(df.index[faux_signal_idx], color='red', linestyle='--', label='Signal MACD')
plt.title('Faux signal : le prix ne monte pas après le croisement MACD')
plt.legend()
plt.show()

**Résumé** :
- Le MACD détecte les accélérations de tendance.
- Les divergences signalent des retournements potentiels.
- Les faux signaux existent, d'où l'intérêt de filtrer avec d'autres indicateurs ou le volume.